In [1]:
from sc3.all import *
import random
import numpy as np

s.boot()

INFO:sc3.synth.server:setting global variable 's' to 'localhost'
INFO:sc3.synth.server:booting server 'localhost' on address 127.0.0.1:57110
INFO:SERVER.stdout:Faust: supercollider.cpp: sc_api_version = 3
INFO:SERVER.stdout:Faust: FaustJPverbRaw numControls=11
INFO:SERVER.stdout:Found 0 LADSPA plugins
INFO:SERVER.stdout:Faust: supercollider.cpp: sc_api_version = 3
INFO:SERVER.stdout:Faust: FaustGreyholeRaw numControls=7
INFO:SERVER.stdout:JackDriver: client name is 'SuperCollider'
INFO:SERVER.stdout:SC_AudioDriver: sample rate = 48000.000000, driver's block size = 1024
INFO:SERVER.stdout:JackDriver: connected  system:capture_1 to SuperCollider:in_1
INFO:SERVER.stdout:JackDriver: connected  system:capture_2 to SuperCollider:in_2
INFO:SERVER.stdout:JackDriver: connected  SuperCollider:out_1 to system:playback_1
INFO:SERVER.stdout:JackDriver: connected  SuperCollider:out_2 to system:playback_2
INFO:SERVER.stdout:SuperCollider 3 server ready.
INFO:SERVER.stdout:JackDriver: max output laten

In [15]:
CmdPeriod.run()

# Synths

In [16]:
@synthdef
def misinte(freq=440, amp=0.1, gate=1, trig:'tr'=1):
    sig = SinOsc(freq) * amp
    sig *= EnvGen(Env.asr(), gate+trig)
    Out(0, sig.dup())

In [17]:
x = misinte(gate=0)

In [13]:
x.set('freq',np.random.rand()*500+100,'trig',1)

# Buses

In [6]:
@synthdef
def misinte2(freq, amp=0.1, gate=1):
    freq = In.kr(freq)
    sig = SinOsc(freq) * amp
    Out(0, sig.dup())
    
@synthdef
def modulator(bus, freq=1.0,amp=1.0,add=0.0):
    Out.kr(bus, SinOsc(freq)*amp+add)

In [7]:
b = ControlBus()
x = misinte2(freq=b)
y = modulator(salida=b)

In [8]:
y.set('add',440,'amp',100.0,'freq',np.random.rand()*50)

In [56]:
CmdPeriod.run()

# Buffer

In [66]:
b1 = Buffer.new_read('../björk  - losss-tIWhpI_unCo.wav')
@synthdef
def player(buff, trig:"tr"=1, rate=1.0, pos=0, dur=1.0, amp=1.0):
    phase = Phasor.ar(trig, BufRateScale.kr(buff) * rate, BufFrames.kr(buff)*pos, BufFrames.kr(buff)*(pos+dur), pos*BufFrames.kr(buff))
    p = BufRd.ar(2,buff,phase)*amp
    Out(0, p)

In [67]:
x = player(b1, 1, 1, pos=0, dur=1,  amp=1)

In [68]:
x.set('rate',0.5,'dur',0.001)

## Reproducir array

In [400]:
n = 2**16
b2 = Buffer(n, channels=2)

In [417]:
sr = s._status_watcher.sample_rate
t = np.arange(n//2)/sr
x = np.sin(2*np.pi*t*440)*np.exp(-t/0.1)
b2.send_list(x.repeat(2))
play(b2)

Synth(temp__50 : 1055)

In [418]:
CmdPeriod.run()

# Sequencer

In [18]:
inf = float('inf')
clock = TempoClock(110/60)

@routine
def r():
    p = Pxrand([0, 2, 4, 7, 9, 12], inf)
    d = Pxrand([1, 0.5, 0.25], inf)
    for note, dur in zip(p, d):
        play(instrument='misinte', note=note,  sustain=dur * random.random())
        yield dur

r.play(clock)

In [19]:
r.stop()